In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import hydra
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torchvision.transforms as transforms
import numpy as np
import os
import pickle as pkl
from pathlib import Path
import cv2
import imageio
import ipdb

from compare_traj import supervisedTraj

In [2]:
# Load the YAML configuration file
with open('/home/lgeng/BAKU/baku/cfgs/config_traj.yaml', 'r') as file:
    config = yaml.safe_load(file)

device = torch.device(config['device'])

In [10]:
supervised_model_path = "/home/lgeng/BAKU/baku/exp_local/eval/2024.08.01_supervised_train/deterministic/153103_hidden_dim_256/supervised_model.pth"
# new checkpoint: /home/lgeng/BAKU/baku/exp_local/eval/2024.07.28_supervised_train/deterministic/225134_hidden_dim_256/supervised_model.pth
# works well: /home/lgeng/BAKU/baku/exp_local/eval/2024.07.16_supervised_train/deterministic/135045_hidden_dim_256/supervised_model.pth
model = supervisedTraj().to(device)

number of parameters: 6.96M


In [11]:
model.load_state_dict(torch.load(supervised_model_path))
model.eval()

supervisedTraj(
  (encoder): GPT(
    (transformer): ModuleDict(
      (wte): Linear(in_features=512, out_features=512, bias=True)
      (wpe): Embedding(256, 512)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-1): 2 x Block(
          (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (attn): CausalSelfAttention(
            (c_attn): Linear(in_features=512, out_features=1536, bias=True)
            (c_proj): Linear(in_features=512, out_features=512, bias=True)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Linear(in_features=512, out_features=2048, bias=True)
            (c_proj): Linear(in_features=2048, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((

In [5]:
data_path = "/home/lgeng/BAKU/expert_demos/libero/libero_single_task_success_failure/KITCHEN_SCENE1_open_the_bottom_drawer_of_the_cabinet_eval_failure_big.pkl"

In [6]:
print(f"Loading {data_path}")
# read
data = pkl.load(open((data_path), "rb"))
observations = data["observations"]
actions = data["actions"]
task_emb = data["task_emb"]
aug = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.ToTensor(),
    ]
    )
# store
episodes = []
for i in range(len(observations)):
    episode = dict(
        observation=observations[i],
        action=actions[i],
        task_emb=task_emb,
    )
    episodes.append(episode)
    # self._max_episode_len = max(
    #     self._max_episode_len,
    #     (
    #         len(observations[i])
    #         if not isinstance(observations[i], dict)
    #         else len(observations[i]["pixels"])
    #     ),
    # )
    # # if obs_type == 'features':
    # self._max_state_dim = max(
    #     self._max_state_dim, data["states"][i].shape[-1]
    # ) if "states" in data.keys() else 0
    # self._num_samples += (
    #     len(observations[i])
    #     if self._obs_type == "features"
    #     else len(observations[i]["pixels"])
    # )

Loading /home/lgeng/BAKU/expert_demos/libero/libero_single_task_success_failure/KITCHEN_SCENE1_open_the_bottom_drawer_of_the_cabinet_eval_failure_big.pkl


In [7]:
# Import necessary Hydra functions
from hydra.core.global_hydra import GlobalHydra
from hydra.experimental import initialize, compose
from omegaconf import OmegaConf
from pathlib import Path

# Check if Hydra is already initialized, and clear it if it is
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()
# Manually initialize Hydra and get the configuration object
initialize(config_path="cfgs")
cfg = compose(config_name="config_traj")

In [8]:
from compare_traj import WorkspaceIL as W

workspace = W(cfg)

# Load weights
snapshots = {}
# bc
bc_snapshot = Path(cfg.bc_weight)
if not bc_snapshot.exists():
    raise FileNotFoundError(f"bc weight not found: {bc_snapshot}")
print(f"loading bc weight: {bc_snapshot}")
snapshots["bc"] = bc_snapshot
workspace.load_snapshot(snapshots)

language_projector = workspace.agent.language_projector
language_projector.eval()
encoder = workspace.agent.encoder
encoder.eval()

workspace: /home/lgeng/BAKU/baku
Loading /home/lgeng/BAKU/expert_demos/libero/libero_single_task_success_failure/KITCHEN_SCENE1_open_the_bottom_drawer_of_the_cabinet_big.pkl
Loading /home/lgeng/BAKU/expert_demos/libero/libero_single_task_success_failure/KITCHEN_SCENE1_open_the_bottom_drawer_of_the_cabinet_failure_big.pkl
Loading /home/lgeng/BAKU/expert_demos/libero/libero_single_task_success_failure/KITCHEN_SCENE1_open_the_bottom_drawer_of_the_cabinet_eval_failure_big.pkl
Loading /home/lgeng/BAKU/expert_demos/libero/libero_single_task_success_failure/KITCHEN_SCENE1_open_the_bottom_drawer_of_the_cabinet_eval_big.pkl



[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python /home/lgeng/miniforge3/envs/baku/lib/python3.9/site-packages/robosuite/scripts/setup_macros.py (macros.py:55)


number of parameters: 6.53M
number of parameters: 25.88M
number of parameters: 25.88M
number of parameters: 25.88M
number of parameters: 6.96M
loading bc weight: /home/lgeng/BAKU/baku/exp_local/2024.06.26_train/deterministic/225305_hidden_dim_256/snapshot/80000.pt


ResnetEncoder(
  (resnet18_base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): GroupNorm(8, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (block_1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): GroupNorm(8, 64, eps=1e-05, affine=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): GroupNorm(8, 64, eps=1e-05, affine=True)
  )
  (block_2): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): GroupNorm(8, 64, eps=1e-05, affine=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): GroupNorm(8, 64, eps=1e-05, affine=True)
  )
  (block_3

In [12]:
indices = []

VIDEO_FILENAME = "videos"
filename_with_extension = os.path.basename(data_path)
filename_without_extension, _ = os.path.splitext(filename_with_extension)
save_dir = os.path.join(os.path.dirname(data_path), VIDEO_FILENAME, filename_without_extension)
os.makedirs(save_dir, exist_ok=True)
height, width = 128, 128

for i, episode in enumerate(episodes):
    traj = episode["observation"]['pixels']
    # Save the video in the created directory
    images = [img.astype('uint8') for img in traj]
    video_path = os.path.join(save_dir, f'{i}.mp4')
    imageio.mimsave(video_path, images, fps=20)
    
    traj_np = traj
    traj = torch.stack(
        [aug(traj[i]) for i in range(len(traj))]
    ).to(device).float()

    task_emb = torch.tensor(task_emb).to(device).float()

    with torch.no_grad():
        lang_features = language_projector(task_emb[None]).repeat(traj.shape[0], 1)
        traj = encoder(traj, lang=lang_features)
    
    traj_batch = traj.unsqueeze(0)
    label_batch = torch.tensor([0]).to(device)
    batch = [traj_batch, label_batch]

    # Predict
    pred = model.show_prob(batch)

    pred_np = pred[0].detach().cpu().numpy()[0]

    # Initialize VideoWriter for each episode
    output_video_path = os.path.join(save_dir, f'output_{i}.mp4')

    # Load the video
    cap = cv2.VideoCapture(video_path)
    frames = []
    for i, frame in enumerate(traj_np):
        # Capture frames in the video 
        ret, frame = cap.read() 
        prob_text = f"Prob: {pred_np[i]:.2f}" if i < len(pred_np) else ""
        # Use putText() method for inserting text on video 
        cv2.putText(frame,  
                    prob_text,  
                    (25, 25),  
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,  
                    (255, 255, 255), 
                    2,  
                    cv2.LINE_4) 
        frames.append(frame)
    # Release the VideoCapture object
    cap.release()
    # Save the video
    imageio.mimsave(output_video_path, frames, fps=20)

    # Get the index of first prob in the pred sequence that is below .5
    below_half = np.where(pred_np < .55)[0]
    indices.append(below_half[0] if len(below_half) > 0 else None)

# close all windows 
cv2.destroyAllWindows() 